#### Mapping to an Explicit Set of Primary Key Columns

The `Mapper` construct in order to _successfully map a table_ __always requires__ that _at least one column_ be identified as the `"primary key"` for that _selectable_. This is so that when an ORM object is loaded or persisted, it _can be placed in the identity map_ with an __appropriate identity key__.

To support this use case, all `FromClause` objects (where `FromClause` is the common base for objects such as `Table`, `Join`, `Subquery`, etc.) have an attribute `FromClause.primary_key` which _returns a collection_ of those `Column` objects that indicate they are part of a __"primary key"__, which is derived from each `Column` object being a member of a `PrimaryKeyConstraint` collection that's associated with the `Table` from which they ultimately derive.

In those cases where the _selectable being mapped_ __does not include columns__ that are _explicitly_ part of the `primary key constraint` on their parent table, a _user-defined_ set of `primary key columns` __must be defined__. The `mapper.primary_key` parameter is used for this purpose.

Given the following example of a `Imperative Table` mapping against an existing `Table` object, as would occur in a scenario such as when the `Table` were __reflected from an existing database__, where the _table does not have any declared primary key_, we may map such a table as in the following example.

In [1]:
from sqlalchemy import Column, MetaData, String, Table, UniqueConstraint
from sqlalchemy.orm import declarative_base

In [2]:
Base = declarative_base()
metadata = MetaData()

In [3]:
group_users = Table(
    "group_users",
    metadata,
    Column("user_id", String(40), nullable=False),
    Column("group_id", String(40), nullable=False),
    UniqueConstraint("user_id", "group_id"),
)

In [4]:
class GroupUsers(Base):
    __table__ = group_users
    __mapper_args__ = {"primary_key": [group_users.c.user_id, group_users.c.group_id]}

Above, the `group_users` table is an _association table_ of some kind with string columns `user_id` and `group_id`, but __no primary key__ is set up; instead, there is only a `UniqueConstraint` establishing that the _two columns represent a unique key_. The `Mapper` __does not automatically inspect unique constraints for primary keys__; instead, we make use of the `mapper.primary_key` parameter, passing a _collection_ of `[group_users.c.user_id, group_users.c.group_id]`, indicating that these _two columns should be used_ in order to __construct the identity key__ for instances of the `GroupUsers` class.